In [ ]:
import numpy as np
import rebound  
import csv

# Start REBOUND simulation
sim = rebound.Simulation()

# Use the IAS15 integrator which is suitable for this type of simulation
sim.integrator = 'ias15'

# Set units for the simulation (years, astronomical units, solar masses)
sim.units = ('yr', 'AU', 'Msun')

# Earth Mass in Solar Units
m_earth = 0.0000030034146856628

# Create particle environment
sim.add(hash="Star", m=1)  # Central Star
sim.add(hash="Terrestrial Planet", m=0.8*m_earth, a=0.8, e=0.02, inc=0)  # Terrestrial Planet
sim.add(hash="Inclined Planet", m=1.6*m_earth, a=1.2, e=0.01, inc=np.radians(75))  # Inclined Planet with 75 degree inclination
sim.add(hash="Gas Giant", m=80*m_earth, a=9.5, e=0.05, inc=0)  # Gas Giant

# Dictionary to store measured quantities
quantities = {'Terrestrial Planet Inclination': [], 'Terrestrial Planet Semi-Major Axis': [],
              'Inclined Planet Inclination': [], 'Inclined Planet Semi-Major Axis': [],
              'Gas Giant Inclination': [], 'Gas Giant Semi-Major Axis': []}

# Time array to store the simulation time steps
time = []

# Collect data values while integrating the simulation for 20 periods of the Inclined Planet
while sim.t <= 20 * sim.particles[2].P:  # sim.particles[2] is the Inclined Planet

    # Measure inclination and semi-major axis for each planet
    for i, planet_hash in enumerate(['Terrestrial Planet', 'Inclined Planet', 'Gas Giant']):
        orbit = sim.particles[i + 1].orbit(primary=sim.particles[0])  # Calculate orbit relative to the star
        quantities[planet_hash + ' Inclination'].append(orbit.inc)
        quantities[planet_hash + ' Semi-Major Axis'].append(orbit.a)

    # Record current simulation time
    time.append(sim.t)
    
    # Integrate the simulation forward by one time step (sim.dt)
    sim.integrate(sim.t + sim.dt, exact_finish_time=False)

    # Move the simulation to the center of mass frame
    sim.move_to_com()
        

# Save the time data to a CSV file
with open('Inclined_Orbit_Data(75degrees_time).csv', 'w', newline='') as csvfile:
    w = csv.writer(csvfile)
    w.writerow(time)

# Save the measured quantities to a separate CSV file
with open("Inclined_Orbit_Data(75degrees_quantities).csv", "w", newline="") as f:
    w = csv.DictWriter(f, quantities.keys())
    w.writeheader()
    w.writerow(quantities)
